In [1]:
%autosave 0
import os
import glob
import numpy 
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np 
import tensorflow as tf
from pandas import read_csv
from matplotlib import pyplot
from pandas.plotting import scatter_matrix 
from numpy import set_printoptions
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRFRegressor, XGBRegressor
from sklearn.compose import ColumnTransformer
from datetime import datetime
from mlflow import log_metric, log_param

set_printoptions(precision=0)
pd.set_option("display.precision", 2)
pd.option_context('display.float_format', '{:0f}'.format);

seed = 15
tf.random.set_seed(seed)
# tf.set_random_seed(seed)
numpy.random.seed(seed)

Autosave disabled


# création du dataset final

récupération de tous les fichiers csv commençant par dataset_final 

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('../csv/dataset_final_*.{}'.format(extension))]

concaténation de tous les fichiers récupérés afin d'en obtenir qu'un seul

In [3]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "../csv/dataset_final.csv", index=False, encoding='utf-8-sig')

# préprocessing du dataset

récupération du fichier csv concaténé précédement

In [4]:
# Importation du fichier csv dans un dataframe
df_train_full = read_csv('../csv/dataset_final.csv', index_col=0)

mise en forme du dataframe

In [5]:
df_train_full = df_train_full.set_index('ref')
df_train_full = df_train_full.drop(columns=['titre'])
print(df_train_full.shape)
df_train_full.head(5)

(8880, 12)


,city,departement,region,type,living_area_m2,lot_size_m2,nb_room,nb_bedroom,pool,cellar,garage,output
ref,,,,,,,,,,,,
/fr/properties/20949a-s2519.htm,Thiviers,Dordogne,Aquitaine,manoir,385.0,3446.0,12.0,7.0,False,True,False,475000
/fr/properties/66377a-4508aail.htm,Casteljaloux,Lot-et-Garonne,Aquitaine,terrain,8610.0,8610.0,NaN,NaN,False,False,False,74000
/fr/properties/58577a-609.htm,Périgueux,Dordogne,Aquitaine,fond de commerce,200.0,NaN,5.0,NaN,False,True,True,386900
/fr/properties/2756a-30293986.htm,Blaye,Gironde,Aquitaine,maison,154.0,901.0,6.0,5.0,False,False,False,170500
/fr/properties/30412a-127963rr.htm,Bougarber,Pyrénées-Atlantiques,Aquitaine,autre,169.0,1500.0,8.0,4.0,False,False,True,265000


In [6]:
df_train_full.describe()

,living_area_m2,lot_size_m2,nb_room,nb_bedroom,output
count,8238.00,6.83e+03,7250.00,7134.00,8.88e+03
mean,321.09,1.69e+04,6.80,4.30,4.12e+05
std,1086.92,7.31e+04,3.73,2.24,6.16e+05
min,1.00,1.00e+00,2.00,2.00,8.00e+03
25%,112.00,9.87e+02,4.00,3.00,1.56e+05
50%,168.00,2.50e+03,6.00,4.00,2.68e+05
75%,260.00,7.00e+03,8.00,5.00,4.57e+05
max,32767.00,1.94e+06,50.00,42.00,1.60e+07


split du dataframe en input et output

In [7]:
Y_trainfull = df_train_full['output']
X_trainfull = df_train_full.drop('output', axis=1)

In [8]:
print(Y_trainfull.shape)
print(X_trainfull.shape)

(8880,)
(8880, 11)


Récupération de la liste des villes

In [34]:
import json

list_ville = X_trainfull["city"].unique()
list_ville

with open("../csv/liste_ville.txt", "w") as output:
    for row in list_ville:
        output.write(str(row) + '\n')

In [35]:
list_departement = X_trainfull["departement"].unique()
list_departement

with open("../csv/list_departement.txt", "w") as output:
    for row in list_departement:
        output.write(str(row) + '\n')

In [36]:
list_region = X_trainfull["city"].unique()
list_region

with open("../csv/list_region.txt", "w") as output:
    for row in list_region:
        output.write(str(row) + '\n')

# Traitement des variables catégorielles

Sélection de toutes les colonnes concernant les variables catégorielles

In [ ]:
cat_columns = ['city', 'departement', 'region', 'type']

création des pipelines onehotencoder afin de transformer les variables catégorielles en quantitatives

In [ ]:
city_pipe = Pipeline(steps=[
    ('one', OneHotEncoder(categories = [X_trainfull['city'].unique()]))
])

In [ ]:
departement_pipe = Pipeline(steps=[
    ('one', OneHotEncoder(categories = [X_trainfull['departement'].unique()]))
])

In [ ]:
region_pipe = Pipeline(steps=[
    ('one', OneHotEncoder(categories = [X_trainfull['region'].unique()]))
])

In [ ]:
type_pipe = Pipeline(steps=[
    ('one', OneHotEncoder(categories = [X_trainfull['type'].unique()]))
])

# Traitement des valeurs manquantes

Détection de la présence de valeurs manquantes

In [ ]:
df_train_full.isna().any()

Sélection des colonnes de valeurs quantitatives

In [ ]:
boolean_columns = ['pool', 'cellar', 'garage']
area_column = ['living_area_m2']
ground_column = ['lot_size_m2']
nb_room_column = ['nb_room']
nb_bedroom_column = ['nb_bedroom']

Création des pipelines gérant les valeurs manquantes dans les colonnes de valeurs quantitatives

In [ ]:
area_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

In [ ]:
ground_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0))
])

In [ ]:
room_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=1))
])

In [ ]:
bedroom_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=1))
])

# Construction du pipeline de pré-traitement

In [ ]:
preprocess_pipe = ColumnTransformer(transformers=[
    ('city', city_pipe, ['city']),
    ('departement', departement_pipe, ['departement']),
    ('region', region_pipe, ['region']),
    ('type', type_pipe, ['type']),
    ('area', area_pipe, area_column),
    ('ground', ground_pipe, ground_column),
    ('room', room_pipe, nb_room_column),
    ('bedroom', bedroom_pipe, nb_bedroom_column),
])

# Choix du modèle et intégration dans la pipeline

Création de la fonction calculant la valeur moyenne absolut en pourcentage d'erreur

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)

Création de la fonction de test de différents modèles

In [ ]:
def mae_model(preprocess_pipe, model):

    full_pipe = Pipeline(steps=[
        ('pp', preprocess_pipe),
        ('model', model)
    ])

    X_train, X_val, Y_train, Y_val = train_test_split(X_trainfull, Y_trainfull,
                                                      test_size = 0.2, random_state = seed)

    full_pipe.fit(X_train, Y_train);

    y_train_pred = full_pipe.predict(X_train)
    y_val_pred = full_pipe.predict(X_val)

    mae_train = round(mean_absolute_percentage_error(Y_train, y_train_pred), 0)
    mae_val = round(mean_absolute_percentage_error(Y_val, y_val_pred), 0)

    return mae_train, mae_val

Liste des différents modèles

In [ ]:
lin = LinearRegression()
#log = LogisticRegression()
tree = DecisionTreeRegressor()
rf = RandomForestRegressor()
bag = BaggingRegressor()
xgb = XGBRegressor()
xgb_rf = XGBRFRegressor()

In [ ]:
models = [lin, tree, rf, bag, xgb, xgb_rf]

calcul du MAPE

In [ ]:
errors = []
for model in models:
    errors += [mae_model(preprocess_pipe, model)]

print(errors)

In [ ]:
plt.plot(errors);
plt.grid();
plt.legend(['MAPE sur train', 'MAPE sur val']);

Du graphique précédent, nous choisissons de conserver le modèle 5 à savoir le modèle xgbregressor

# Génération du modèle retenu

In [ ]:
model = XGBRegressor()

In [ ]:
full_pipe = Pipeline(steps=[
        ('pp', preprocess_pipe),
        ('model', model)
    ])

In [ ]:
full_pipe.fit(X_trainfull, Y_trainfull);

## exportation du modèle

In [ ]:
# suppression du fichier si un ancien modèle existe
if os.path.exists('../models/full_pipe.file'):
    os.remove('../models/full_pipe.file')
else:
    print("Impossible de supprimer le fichier car il n'existe pas")

In [ ]:
model_file_name = "../models/full_pipe_" + str(datetime.today())+ ".file"
last_model_name = "../models/full_pipe.file"

In [ ]:
pickle.dump(full_pipe, open(model_file_name, "wb" ))
pickle.dump(full_pipe, open(last_model_name, "wb" ))

# Prédictions

## chargement du modèle

In [ ]:
full_pipe = pickle.load(open(last_model_name, "rb"))

In [ ]:
type(full_pipe)

## prédiction sur trainfull complet

In [ ]:
X_trainfull

In [ ]:
y_train_pred = full_pipe.predict(X_trainfull)

In [ ]:
mape_train = round(metrics.mean_absolute_percentage_error(Y_trainfull, y_train_pred), 0)
print('MAPE sur train : ', mape_train, '€')

In [ ]:
trainfull_pred = pd.DataFrame(y_train_pred)
trainfull_pred.to_csv('../csv/trainfull_pred.csv', index=False)
Y_trainfull.to_csv('../csv/Y_trainfull.csv', index=False)

## prédiction sur valeur unique

In [ ]:
new_value = pd.DataFrame(data = [
    ['Bordeaux', 'Gironde', 'Aquitaine', 'appartement', 54, 12, 1, 1, 'True', 'True', 'True']
    ], index = ['test'], columns = X_trainfull.columns)

new_value.index.name = 'ref'
new_value

In [ ]:
full_pipe.predict(new_value)

# Enregistrement des résultats si voulus (mettre log = True)
-> accessible sous localhost:3000

In [ ]:
log_param("X_trainfull.shape", X_trainfull.shape)
log_metric("MAE sur train", mae_train)